In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [ ]:
filename = "all_arxiv_titles.csv"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

#simple embedding scheme
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)

seq_length = 25
dataX = []
dataY = []

for line in raw_text.splitlines()[:]:
    for i in range(0, len(line)- seq_length, 1):
        prefix = line[i: i + seq_length]
        suffix = line[i+seq_length]
        dataX.append([char_to_int[char] for char in prefix])
        dataY.append([char_to_int[suffix]])
        
n_patterns = len(dataX)

X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X /float(n_vocab)
y = torch.tensor(dataY)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from sklearn.model_selection import train_test_split

class CharModel(nn.Module):
    def __init__(self, hidden_size=128):
        super(CharModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # take only the last output
        x = self.linear(self.dropout(x))  # produce output
        return x

# Parameters
n_epochs = 40
batch_size = 128
hidden_size = 128

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoaders
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=False, batch_size=batch_size)

# Initialize model, optimizer, and loss function
model = CharModel(hidden_size=hidden_size)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")

best_model = None
best_loss = np.inf

# Training loop
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            test_loss += loss.item()
            
    test_loss /= len(test_loader.dataset)
    
    if test_loss < best_loss:
        best_loss = test_loss
        best_model = model.state_dict()
        
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

# Save the best model and char_to_int dictionary
torch.save([best_model, char_to_int], "single-char.pth")
